In [1]:
#Import required Libraries 
import heapq
import os

#Class for Binary Tree Node
class BinaryTreeNode:
    #Constructor
    def __init__(self,value,freq):
        #Binary Tree Node will have character, frequency of occurance and left and right child
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    #We have to compare Binary Tree Node on basis of frequency so we have to overload functions
    def __lt__(self,other): #Less than
        return self.freq < other.freq #Node is less if frequency is less than other
    def __eq__(self,other): #Equal to
        return self.freq == other.freq #Node is equal if frequency is equal to other

#Class for Huffman Coding
class HuffmanCoding:
    #Constructor
    def __init__(self,path):
        self.path = path #File path
        self.__heap = [] #min-Heap
        self.__codes = {} #Codes
        self.__reverseCodes = {} #Reverse codes
    
    #Make frequency dictionary using the text
    def __make_frequency_dict(self,text):
        freq_dict = {}
        for char in text:
            if char not in freq_dict:
                freq_dict[char] = 0
            freq_dict[char] += 1
        return freq_dict
    
    #Construct min-Heap from the frequency dictionary
    def __buildHeap(self,freq_dict):
        for key in freq_dict:
            frequency = freq_dict[key] #Get frequency
            binary_tree_node = BinaryTreeNode(key,frequency) #Create Binary Tree Node
            heapq.heappush(self.__heap,binary_tree_node) #Push Binary Tree Node into min-Heap
            
    #Construct the Binary Tree from the min-Heap
    def __buildTree(self):
        #Everytime Tree will require two minimum nodes
        while len(self.__heap) > 1: #Last node will stay in heap (root for __buildCodes()) 
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            #Non Leaf node will have no value and sum of left and right child frequency
            freq_sum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTreeNode(None,freq_sum)
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            heapq.heappush(self.__heap,newNode)
        return
    
    #Helper function for __buildCodes()
    def __buildCodesHelper(self,root,curr_bits):
        #If we move to left/right child and left/right child doesn’t exist
        if root is None: #Edge Case
            return
        #Only Leaf node will have value in Huffman Coding Binary Tree
        if root.value is not None: #Base Case
            #Codes are stored in this list which will later be required for Encoding
            self.__codes[root.value] = curr_bits
            #Reverse codes are stored in this list which will later be required for Decoding
            self.__reverseCodes[curr_bits] = root.value
            return
        #Traverse recursively in Binary Tree
        self.__buildCodesHelper(root.left,curr_bits+"0") #Traverse left add 0
        self.__buildCodesHelper(root.right,curr_bits+"1") #Traverse right add 1
    
    #Construct the Codes from the Binary Tree
    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.__buildCodesHelper(root,"")
    
    #Create the Encoded text using Codes
    def __getEncodedText(self,text):
        encoded_text = "" #Initially will be empty
        for char in text: #Iterate through each char of text and convert to code
            encoded_text+=self.__codes[char]
        return encoded_text
    
    #Pad the Encoded text
    def __getPaddedEncodedText(self,encoded_text):
        padded_amount = 8-(len(encoded_text)%8) #Calculate required padding
        for i in range(padded_amount): #Pad necessary zeros at end to make multipe of 8
            encoded_text+='0'
        padded_info = "{0:8b}".format(padded_amount) #Will convert binary into 8 bits by adding zero if required at front
        padded_encoded_text = padded_info + encoded_text #Will add how much encoding is done at start
        return padded_encoded_text
    
    #Convert 8 bits to 1 byte
    def __getBytesArray(self,padded_encoded_text):
        array = []
        for i in range(0,len(padded_encoded_text),8):
            byte = padded_encoded_text[i:i+8]
            array.append(int(byte,2)) #Convert byte string to integer taking base as 2 for binary
        return array
    
    #Compress the given file
    def compress(self):
        #Get file from path
        file_name,file_extension = os.path.splitext(self.path) #splitext splits on basis of . (sample.txt -> sample and txt)
        output_path = file_name + ".bin" #Store output bin file at same location
        #Read text from file
        #Open input file in read format and output file in write binary format
        with open(self.path,'r+') as file, open(output_path, 'wb') as output:
            text = file.read() #File read
            text = text.rstrip() #Removes trailing spaces
            #Make frequency dictionary using the text
            freq_dict = self.__make_frequency_dict(text)
            #Construct min-Heap from the frequency dictionary
            self.__buildHeap(freq_dict)
            #Construct the Binary Tree from the min-Heap
            self.__buildTree()
            #Construct the Codes from the Binary Tree
            self.__buildCodes()
            #Create the Encoded text using Codes
            encoded_text = self.__getEncodedText(text)
            #Pad the Encoded text
            #Padding is necessary beacuse Encoded Text will be stored as multiple of 8 bits
            #This extra zeros to make multiple of 8 bits will be stored at start to keep track
            padded_encoded_text = self.__getPaddedEncodedText(encoded_text)
            #Put this Encoded text into the Binary file
            #Convert 8 bits to 1 byte
            bytes_array = self.__getBytesArray(padded_encoded_text)
            #Return this Binary file as output
            final_bytes = bytes(bytes_array) #Using python's inbuilt function convert to bytes
            output.write(final_bytes) #Write to output file
        print('Compressed')
        return output_path
    
    #Remove padding
    def __removePadding(self,text):
        padded_info = text[:8] #Padded info is present at first 8 index
        extra_padding = int(padded_info,2) #Get extra padding info in integer form
        text = text[8:] #Actual text is from 8th index
        text_after_padding_removed = text[:-1*extra_padding]
        return text_after_padding_removed
    
    def __decodeText(self,text):
        decoded_text = ""
        current_bits = ""
        for bit in text:
            current_bits+=bit #Keep adding code
            #As soon as current_bits code matches reverseCodes append character corresponding to the code in decoded_text
            if current_bits in self.__reverseCodes:
                character = self.__reverseCodes[current_bits] #Get character corresponding to the code in decoded_text
                decoded_text+=character
                current_bits = "" #Reset current_bits to empty
        return decoded_text
    
    #Decompress the given file
    def decompress(self,input_path):
        #Get file from path
        file_name,file_extension = os.path.splitext(self.path) #splitext splits on basis of . (sample.txt -> sample and txt)
        output_path = file_name + "_decompressed" + ".txt" #Store output decompressed file at same location
        #Read text from file
        #Open input file in read binary format and output file in write format
        with open(input_path,'rb') as file, open(output_path, 'w') as output:
            bit_string = "" #Get same earlier binary string
            byte = file.read(1) #Read byte one by one
            while byte: #Read byte one by one till byte becomes NULL
                byte = ord(byte)
                # rjust to make it into 8 bits format
                bits = bin(byte)[2:].rjust(8,'0') #ord output is such that from index 2 is bits 
                bit_string += bits
                byte = file.read(1)
            #Remove padding
            actual_text = self.__removePadding(bit_string)
            #Decode Text
            decompressed_text = self.__decodeText(actual_text)
            output.write(decompressed_text) #Write to output file
        return

In [3]:
path = ''
h = HuffmanCoding(path)
output_path = h.compress()
h.decompress(output_path)

FileNotFoundError: ignored